In [21]:
import pickle,sys,os,lda,scipy,pandas
import numpy as np
from nltk import word_tokenize, sent_tokenize
bios = []
for suf in ['1000']:
    bios += pandas.read_pickle("train-corpus/corpus"+suf+".pickle")

In [22]:
def progress(i, end_val, bar_length=50):
    percent = float(i) / end_val
    hashes = '#' * int(round(percent * bar_length))
    spaces = ' ' * (bar_length - len(hashes))
    sys.stdout.write("\r{0} / {1} Percent: [{2}] {3}%".format(i, end_val, hashes + spaces, int(round(percent * 100))))
    sys.stdout.flush()

In [23]:
bios[0]

[('Arne Kaijser',
  [('Biography',
    "Arne Kaijser (born 1950) is a professor of History of Technology at the Royal Institute of Technology in Stockholm, and the head of the university's department of History of science and technology.\n\nKaijser has published two books in Swedish: ''Stadens ljus. Etableringen av de forsta svenska gasverken'' and ''I fadrens spar. Den svenska infrastrukturens historiska utveckling och framtida utmaningar'', and has co-edited several anthologies. Kaijser is a member of the Royal Swedish Academy of Engineering Sciences since 2007 and also a member of the editorial board of two scientific journals: ''Journal of Urban Technology'' and ''Centaurus''. Lately, he has been occupied with the history of Large Technical Systems.\n\n")])]

## Collect paragraph data for every biography

In [24]:
bios_df = list()
all_tokens = set()
i = 0
for bio in bios:
    data = {
        'person' : bio[0][0],
        'tokenized_paragraphs' : list(),
        'paragraph_splits' : list(),
        'word_splits': [0],
        'length' : 0,
        'segments' : 0
    }
    number_of_words = 0
    for segment in bio[0][1]:
        number_of_paragraphs = 0
        data['segments'] += 1
        for paragraph_text in segment[1].split('\n'):
            tokens = word_tokenize(paragraph_text)
            if len(tokens) > 0:
                number_of_paragraphs += 1
                number_of_words += len(tokens)
                all_tokens |= set(tokens)
                data['tokenized_paragraphs'].append(tokens)  
        data['paragraph_splits'].append(number_of_paragraphs)
        data['word_splits'].append(number_of_words)
    data['length'] = number_of_words
    bios_df.append(data)
    i += 1
    progress(i, len(bios))

1001 / 1001 Percent: [##################################################] 100%

In [25]:
bios_data = pandas.DataFrame(bios_df)
bios_data[:2]

,length,paragraph_splits,person,segments,tokenized_paragraphs,word_splits
0,126,[2],Arne Kaijser,1,"[[Arne, Kaijser, (, born, 1950, ), is, a, prof...","[0, 126]"
1,669,"[4, 13]","Albert Alcibiades, Margrave of Brandenburg-Kul...",2,"[[Albert, II, (, Albrecht, ;, 28, March, 1522,...","[0, 71, 669]"


In [26]:
all_tokens_list = list(all_tokens)
number_of_tokens = len(all_tokens_list)
all_paragraphs = bios_data['tokenized_paragraphs'].sum()
paragraphs_bow = np.zeros([len(all_paragraphs),number_of_tokens], dtype = np.int)
tokens_indices_dict = dict()
for i in range(number_of_tokens):
    tokens_indices_dict[all_tokens_list[i]] = i
    
for i in range(len(all_paragraphs)):
    for w in all_paragraphs[i]:
        paragraphs_bow[i][tokens_indices_dict[w]] += 1
        
    progress(i + 1, len(all_paragraphs))

14031 / 14031 Percent: [##################################################] 100%

## Assign LDA topics to paragraphs with word embeddings

In [27]:
vocab = list(all_tokens)
paragraphs_bow.shape

(14031, 59063)

In [28]:
#model = lda.LDA(n_topics=20, n_iter=1500, random_state=1)
#model.fit(paragraphs_bow)  # model.fit_transform(X) is also available

In [ ]:
#with open('toy_topics.pkl','wb') as toy:
#    pickle.dump(model,toy)

In [29]:
model = pandas.read_pickle('toy_topics.pkl')

In [30]:
vecs = pickle.load(open('/home/ilay/vecs.pkl','rb'))

In [31]:
def paragraph_to_vector(paragraph_tokens):
    l = len(vecs['queen']) # len of the vector is 300
    paragraph_accumulative = np.zeros(l)
    topic_ratings = []
    # just sum the paragraph words' vectors to get a semantic average of it
    for ind,word in enumerate(paragraph_tokens):
        if word in vecs:
            paragraph_accumulative += vecs[word]
    return paragraph_accumulative

### Generate topic lists

In [32]:
topic_word = model.topic_word_ 
n_top_words = 20

topics_words = list()

word_freqs = dict()

for i, topic_dist in enumerate(topic_word):
    # topic_words: words sorted by relevance to topic in descending order
    topic_words = list(np.array(vocab)[np.argsort(topic_dist)[::-1]])#[:10]#[:-(n_top_words+1):-1]
    #print(topic_words)
    topics_words.append(topic_words)
    print('Topic {}: {}'.format(i, ' '.join(topic_words[:15])))
    for word in topic_words[:20]:
        if word not in word_freqs:
            word_freqs[word] = 1
        else:
            word_freqs[word] += 1

Topic 0: walks cartographical ever-flowing Shu cross-dressing sociologist Cuckoo Geoffroy Mysorean ''Quarterly historic unresponsive subtleties 19521962 combinatorial
Topic 1: ever-flowing Geoffroy walks Shu sociologist cross-dressing fourth Marguerite 1.04 obeyed ''Quarterly proposal Marinetti Cuckoo Granada
Topic 2: walks Shu Armored sociologist cross-dressing Smaragdus Cuckoo ever-flowing Jullian Geoffroy subtleties Marguerite fourth counterbalanced 19521962
Topic 3: ever-flowing Geoffroy walks Shu sociologist cross-dressing Marguerite Cuckoo unresponsive Mysorean fourth proposal subtleties Cooper compulsory
Topic 4: walks Shu ever-flowing cartographical Peckham historic fourth Geoffroy Cuckoo sale cross-dressing sociologist tenants Donatism aegis
Topic 5: ever-flowing fourth walks Shu Geoffroy cross-dressing Cuckoo obeyed Marguerite Peckham sociologist Marinetti Shaa expectations 19521962
Topic 6: cartographical walks 3.9 Austrians fatigues orations Shaa Improvised Jailhouse curryk

In [ ]:
topic_words_indices_dicts = list()
for topic_words in topics_words:
    topic_words_indices_dict = dict()
    for i in range(len(topic_words)):
        topic_words_indices_dict[topic_words[i]] = i
    topic_words_indices_dicts.append(topic_words_indices_dict)

def calculate_coherence_score(sentences_topics_frequencies, number_of_topics, position, window_size):
    pre_topics_frequencies = np.zeros(number_of_topics)
    for i in range(window_size):
        pre_topics_frequencies += sentences_topics_frequencies[position - i]
        
    post_topics_frequencies = np.zeros(number_of_topics)
    for i in range(window_size):
        post_topics_frequencies += sentences_topics_frequencies[position + 1 + i]
    return scipy.spatial.distance.cosine(pre_topics_frequencies, post_topics_frequencies) 

def calculate_depth_scores(coherence_scores):
    depth_scores = list()
    for i in range(len(coherence_scores)):
        hl = coherence_scores[i]
        hr = coherence_scores[i]
        for j in range(i):
            if coherence_scores[j] > coherence_scores[i]:
                hl = coherence_scores[j]
            else:
                break
        
        for j in range(i + 1, len(coherence_scores)):
            if coherence_scores[j] > coherence_scores[i]:
                hr = coherence_scores[j]
            else:
                break
                
        depth_score = 0.5 * (hl + hr - 2 * coherence_scores[i])
        depth_scores.append(depth_score)
        
    return depth_scores

In [ ]:
def calculate_word_topic_id(word, topics_words):
    if word in word_freqs and word_freqs[word] > 1:
        return -1
    
    topic_id = -1
    min_topic_index = number_of_tokens + 1
    for current_topic_id in range(number_of_topics):
        index = topic_words_indices_dicts[current_topic_id][word]
        if index < min_topic_index:
            topic_id = current_topic_id
            min_topic_index = index
            
    return topic_id

In [33]:
topic_vectors = list()
for topic_words in topics_words:
    words_taken = 0
    i = 0
    vector = np.zeros(300)
    # take first 200 out of the 59,000 words
    while(words_taken < 200):
        word = topic_words[i]
        if (word not in word_freqs or word_freqs[word] < 5) and word in vecs:
            vector += vecs[word]
            words_taken += 1
        i += 1
    topic_vectors.append(vector)

In [34]:
# make a list of topics for each paragraph by distance of topic vectors from the paragraph vector
def paragraph_topics_rating(paragraph,topic_vectors):
    cosine = scipy.spatial.distance.cosine
    return np.argsort([cosine(paragraph_to_vector(paragraph),topic_vector) for i, topic_vector in enumerate(topic_vectors)])

In [35]:
paragraph_topics_rating(bios_data.loc[1,'tokenized_paragraphs'][0],topic_vectors)

array([ 6,  7, 14, 15, 12,  2, 11, 16,  3,  1,  0, 13,  9,  4,  5, 19, 17,
       10, 18,  8])

This means topic 6 is most strongly linked to this pargraph by the aggregate vectors assessment, then topic 7, etc.

## Split the text by according to the topic ratings.

In [36]:
psplits = list()
wsplits = list()
for i in range(len(bios_data)):
    last_paragraph_topics = np.array([])
    number_of_paragraphs = 1
    number_of_words = 0
    psplit = list()
    wsplit = list()
    for tp in bios_data.loc[i,'tokenized_paragraphs']:
        number_of_words += len(tp)
        paragraph_topics = paragraph_topics_rating(tp,topic_vectors)[:3]
        if len(last_paragraph_topics) > 0:
            if len(np.intersect1d(paragraph_topics, last_paragraph_topics)) > 0:
                number_of_paragraphs += 1
            else:
                psplit.append(number_of_paragraphs)
                wsplit.append(number_of_words)
                number_of_paragraphs = 1
        else:
            wsplit.append(0)
        last_paragraph_topics = paragraph_topics
        
       
    if number_of_paragraphs > 0:
        psplit.append(number_of_paragraphs)
        wsplit.append(number_of_words)
    psplits.append(psplit)
    wsplits.append(wsplit)
bios_data['tst_word_splits'] = pandas.Series(wsplits,index=bios_data.index)
bios_data['tst_paragraph_splits'] = pandas.Series(psplits, index=bios_data.index)

In [ ]:
from scipy.stats import entropy as KL
def paragraph_pairs(bio):
    stats = list()
    
    number_of_paragraphs = 1
    for i in range(len(bio) - 1):
        paragraph1 = bio[i]
        paragraph2 = bio[i + 1]
        shulman_distance = Shulman_Distance(paragraph_topics_rating(paragraph1,topic_vectors),
                               paragraph_topics_rating(paragraph2,topic_vectors))
        print((i + 1), " ".join(bio[i][:10]))
        print()
        print(shulman_distance)
        print()
    
        last_shulman_distance = shulman_distance
        
import scipy    
def flip_array(a):
    flipped = [0] * len(a)
    #print(len(flipped))
    for i in range(len(a)):
        flipped[a[i]] = i
    return np.array(flipped)
        
def Shulman_Distance(v1,v2):
    v1flip = flip_array(v1)
    v2flip = flip_array(v2)
    diff = v1flip - v2flip
    return scipy.linalg.norm(diff)

In [38]:
bios_data[:5]

,length,paragraph_splits,person,segments,tokenized_paragraphs,word_splits,tst_word_splits,tst_paragraph_splits
0,126,[2],Arne Kaijser,1,"[[Arne, Kaijser, (, born, 1950, ), is, a, prof...","[0, 126]","[0, 126]",[2]
1,669,"[4, 13]","Albert Alcibiades, Margrave of Brandenburg-Kul...",2,"[[Albert, II, (, Albrecht, ;, 28, March, 1522,...","[0, 71, 669]","[0, 669]",[17]
2,239,[4],Aimoin,1,"[[Aimoin, (, circa, 960, c., 1010, ), ,, Frenc...","[0, 239]","[0, 239]",[4]
3,314,[4],Alcamenes,1,"[[Alcamenes, (, lang-grc|, ), was, an, ancient...","[0, 314]","[0, 314]",[4]
4,270,"[1, 4]",Alexander Balas,2,"[[Alexander, Balas, (, o, B, ), ,, ruler, of, ...","[0, 17, 270]","[0, 93, 270]","[1, 4]"


## Compare with Alexaner A Alemi and Paul Ginsparg's Method
We took the code (https://github.com/alexalemi/segmentation.git) described in this article:
http://arxiv.org/pdf/1503.05543v1.pdf  and modified it a little to fit our available embeddings DB and the presentation needs. Running it on the data gives pretty poort results, but can serve as basis for evaluation of our own method.

In [39]:
sys.path.append('segmentation/code')
from segmentation.code.segmentart import *

In [41]:
from nltk.metrics.segmentation import *
def splits_list(bio,ind,acc):
    if ind == len(bio)-1:
        return acc
    elif ind == 0:
        acc.append(len(bio[ind][1].split()))
    else:
        acc.append(acc[ind-1]+len(bio[ind][1].split()))
    return splits_list(bio,ind+1,acc)

def indexlist2binary(index_list):
    ret = "1"
    for ordinal,split_location in enumerate(index_list):
        if ordinal == 0:
            continue
        ret += "0"*(split_location - index_list[ordinal - 1])
        ret += "1"
    return ret

alexmi = []
ours = []
for i in range(len(bios_data)):
    onepiece = " ".join([" ".join(tp) for tp in bios_data.loc[i,'tokenized_paragraphs']])
    gld = bios_data.loc[i,'word_splits']
    tst = [0] + segmentize(onepiece,bios_data.loc[i,'segments'],vecs) + [bios_data.loc[i,'length']] if len(gld) > 2 else gld
    if len(tst)*len(gld) > 0:
        alexmi.append({
            'person' : bios_data.loc[i,'person'], 
            'alexmi pk' : pk(indexlist2binary(gld),indexlist2binary(tst))
        })
        ours.append({
                'person': bios_data.loc[i,'person'],
                'our pk' : pk(indexlist2binary(gld),indexlist2binary(bios_data.loc[i,'tst_word_splits']))
            })
    progress(i, len(bios))    

1000 / 1001 Percent: [##################################################] 100%

In [42]:
alexmi = pandas.DataFrame(alexmi)
ours = pandas.DataFrame(ours)
alexmi.describe()

,alexmi pk
count,1001.000000
mean,0.345076
std,0.235191
min,0.000000
25%,0.000000
50%,0.415364
75%,0.523810
max,0.780822


In [43]:
ours.describe()

,our pk
count,1001.000000
mean,0.218856
std,0.174678
min,0.000000
25%,0.019417
50%,0.207207
75%,0.327860
max,0.903361


So we can see that even though we let the Alexmi code off all one-segment biographies, our method's mean is still better.
Sadly, it's probably not good enough for feeding to the segment classifier for getting any reasonable results.